> ## Make a copy of this notebook (File menu -> Make a Copy...)

##### Throughout this week's homework, use your LUdet(A) function that you wrote in lab.

### Homework Question 1

1. Implement a function called *Cramers(A,v)* that solves the matrix equation $Ax=v$ using Cramer's Rule. If you need a reminder of Cramer's Rule, [see here](https://en.wikipedia.org/wiki/Cramer%27s_rule).<br><br>

1. Consider the matrix equation $Ax=v$ where <br><br>
   
   $$A=\begin{bmatrix}-4 & 1 & -9 &  4 &  4 &  1 &  9 & -6\\
-1 & -3 & -8 & -2 & -4 &  4 &  2 &  7\\
 3 &  6 & -8 &  1 &  0 &  1 &  2 &  5\\
-10 & -4 & -1 & -4 &  2 & -8 &  3 &  2\\
-6 &  7 & -4 & -6 &  2 &  1 & -4 &  8\\
 6 &  6 &  5 &  8 & -7 &  9 & -1 &  9\\
 1 & -10 &  2 & -6 & -8 & -9 & -4 &  0\\
 6 &  1 & -4 &  3 &  8 &  3 &  3 &  5\end{bmatrix}\mbox{ and }v=\begin{bmatrix} 28 \\ 40 \\ 45 \\ -18 \\ 26 \\ 114 \\ -125 \\ 102\end{bmatrix}$$<br><br> 
 
      Solve these using both your Cramer's Rule function and previous solution techniques. Run timing comparisons and comment on your results. For your convenience, here is code for those arrays:
      
```python
A = np.array([[4 , 1 , -9 ,  4 ,  4 ,  1 ,  9 , -6],[1 , -3 , -8 , -2 , -4 ,  4 ,  2 ,  7],[3 ,  6 , -8 ,  1 ,  0 ,  1 ,  2 ,  5],[10 , -4 , -1 , -4 ,  2 , -8 ,  3 ,  2],[6 ,  7 , -4 , -6 ,  2 ,  1 , -4 ,  8],[6 ,  6 ,  5 ,  8 , -7 ,  9 , -1 ,  9],[1 , -10 ,  2 , -6 , -8 , -9 , -4 ,  0],[6 ,  1 , -4 ,  3 ,  8 ,  3 ,  3 ,  5]])

v=np.array([28,40,45,-18,26,114,-125,102])
```
      

In [45]:
import numpy as np
from rref import rref 

def rowaddmult(A,i,j,c):
    A[j] = A[j] + (c * A[i])
    return A

def swaprows(A,i,j):
    A[[i,j]] = A[[j,i]]
    return A

def rowredpivot(a):
    copy = a.copy().astype("float")
    rows,cols = copy.shape
    for i in range(rows):
        slice = copy[i:,i] #look at the current column from the current row
        slice = np.absolute(slice) #Take the absolute value of all the entries
        swaprows(copy,i,np.argmax(slice)+i)
        for j in range(i+1,rows): #row reduce by multiply by scalars and adding
            rowaddmult(copy,i,j,(-1*(copy[j,i]/copy[i,i])))
    return copy

def backsub(U,v):
    rows,cols = U.shape
    res = np.zeros(cols)
    for row in range(rows-1,-1,-1):
        res[row] = (v[row] - res[row:]@U[row,row:])/U[row,row]
    return res

def rowred(a):
    copy = a.copy().astype("float")
    rows,cols = copy.shape
    for i in range(rows):#tracks the current row
        for j in range(i+1,rows):#performs operations on the rows below the current one
            rowaddmult(copy,i,j,(-1*(copy[j,i]/copy[i,i])))
    return copy

def rowredinv(a):
    copy = a.copy().astype("float")
    rows,cols = copy.shape
    identity = np.identity(rows)
    a_augment_i = np.zeros((rows,cols*2))
    a_augment_i[:,:cols] = copy
    a_augment_i[:,cols:] = identity
    reduced = rref(a_augment_i,10**-10)
    return reduced[:,cols:]

def det(A):
    rows,cols = A.shape
    deter = 0
    if rows != cols:
        raise ValueError("The matrix is not square")
    elif rows == 2:
        return det2(A)
    elif rows == 3:
        return det3(A)
    else:
        for i in range(rows):
            deter += ((-1)**(i+2))*A[0][i]*det(minor(A,1,i+1))
        return deter

def minor(A,i,j):
    rows,cols = A.shape
    res = np.zeros((rows-1,cols-1))
    
    res[:i-1,:j-1] = A[:i-1,:j-1]
    res[:i-1,j-1:] = A[:i-1,j:]
    res[i-1:,:j-1] = A[i:,:j-1]
    res[i-1:,j-1:] = A[i:,j:]
    
    return res

def det3(A):
    accum = 0
    for i in range(3):
        accum += ((-1)**(2+i)) * A[0][i]*det2(minor(A,1,i+1))
    return accum
def det2(A):
    row, col = A.shape
    if row != 2 or col !=2:
        raise ValueError("The matrix is not a 2x2")
    return A[0][0]*A[1][1] - A[0][1]*A[1][0]

def LU(A):
    U = A.copy().astype("float")
    rows,cols = U.shape
    L = np.zeros_like(U)
    P = np.eye(rows)
    swapCounter = 0
    for i in range(rows):
        slice = U[i:,i] #look at the current column from the current row
        slice = np.absolute(slice) #Take the absolute value of all the entries
        index = np.argmax(slice)+i
        if i != index: #if the index of the current row isn't the index of the of the max entry, swap the two rows
            swaprows(U,i,index)
            swaprows(L,i,index)
            swaprows(P,i,index)
            swapCounter += 1
        for j in range(i+1,rows): #row reduce by multiply by scalars and adding
            holder = U[j,i]/U[i,i]
            L[j,i] = holder
            rowaddmult(U,i,j,(-1*holder))
    np.fill_diagonal(L,1.)         
    return (L, U, P, swapCounter)

def LUdet(A):
    L,U,P,swapCounter = LU(A)
    return U.diagonal().prod()/((-1)**swapCounter)


def Cramers(A,v):
    # calculate det(A) using LUdet(A)
    D = LUdet(A)
    x_values = []  #create array to store x's 
    
    for i in range(A.shape[0]):
        B = A.copy() 
        B[:,i] = v 
        x_values.append(LUdet(B)/D)
        
        
    return x_values

#test array 
A = np.array([[-4 , 1 , -9 , 4 , 4 , 1 , 9 , -6],
[-1 , -3 , -8 , -2 , -4 , 4 , 2 , 7],
[3 , 6 , -8 , 1 , 0 , 1 , 2 , 5],
[-10 , -4 , -1 , -4 , 2 , -8 , 3 , 2],
[-6 , 7 , -4 , -6 , 2 , 1 , -4 , 8],
[6 , 6 , 5 , 8 , -7 , 9 , -1 , 9],
[1 , -10 , 2 , -6 , -8 , -9 , -4 , 0],
[6 , 1 , -4 , 3 , 8 , 3 , 3 , 5]]).astype(float)

v=np.array([28,40,45,-18,26,114,-125,102])

print(Cramers(A,v))
%timeit LUdet(A)
%timeit Cramers(A,v)
%timeit det(A)

[-2.1055912665912414e-15, 1.0000000000000002, 1.999999999999991, 2.999999999999994, 3.9999999999999982, 5.0, 6.0, 7.000000000000002]
286 µs ± 3.81 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.68 ms ± 69 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
191 ms ± 2.71 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Time complexity analysis: 
We found that the fastest function was LUdet() which ran in 316 µs. The second fastest function was Cramers() which ran in 2.69 ms. The slowest function was det() as expected which ran in 194 ms. This was expected as the body of the Cramers() calls LUdet(), so naturally Cramers() would have to be slower than LUdet(). Likewise, we expected det() to be the slowest function as it requires the most computation with its recursive algorithm.

### Homework Question 2

The inverse $A^{-1}$ of a matrix can be found using the *cofactor matrix*. If $A$ is a matrix<br><br> $$A=\begin{bmatrix}a_{1,1} & \cdots & a_{1,n} \\ \vdots & \ddots & \vdots \\ a_{n,1} & \cdots & a_{n,n}\end{bmatrix}$$<br><br> then its *cofactor matrix* is defined to be $$adj(A)=\begin{bmatrix}C_{1,1} & \cdots & C_{1,n} \\ \vdots & \ddots & \vdots \\ C_{n,1} & \cdots & C_{n,n}\end{bmatrix}^T,$$<br><br> where $C_{i,j}$ is the $(i,j)$ cofactor of $a_{i,j}$ (note the transpose sign!).<br><br>

1. Find by hand the cofactor matrix of <br><br>$$A=\begin{bmatrix} -2 & 11 & 48 \\-2 & 11 & 49 \\3 & -16 & -70
\end{bmatrix}.$$<br><br>
1. Find the determinant of $A$, and show that $A\cdot adj(A)=det(A)\cdot I_3$ in this case (where $I_3$ is the $3\times 3$ identity matrix).<br><br>
1. This property is true in general. Use this to write a function called `detInv(A)` that computes the inverse of $A$ using determinants. Try to avoid computing cofactors twice!<br><br>
1. Use your function to compute the inverse of the matrix from the first part of this question above.<br><br>
1. Is this an efficient way to compute inverses? Experimentally show that once again, using row reduction is far better. (See [here](https://www.mathsisfun.com/algebra/matrix-inverse-row-operations-gauss-jordan.html) if you need a reminder as to how to find inverses using row reduction. Feel free to use the `rref(A,tol)` function.)

Question 5: 
This is NOT an efficient way to compute inverses. This is because first you need to compute the cofactor matrix which involves a double-nested for-loop which is computation intensive. You also have to calculate the determinant and perform row operations to calculate the PA=LU decomposition. Meanwhile, computing the inverse via row reduction only requires row operations and does not require further computation such as calculating the cofactor matrix or determinant. In short, this method of computing the inverse involves much more computation than is necessary. 

In [57]:
#Question 1: We found that our cofactor matrix was the same when computed by hand and 
#via our cofactor algorithm. The answer we got for both is: 
A = np.array([[-2,11,48],[-2,11,49],[3,-16,-70]])

print("Cofactor matrix of: ",end="\n\n")
print(C,end="\n\n")
print("is:",end="\n\n")
print(cofactor(C),end="\n\n")

Cofactor matrix of: 

[[ -2  11  48]
 [ -2  11  49]
 [  3 -16 -70]]

is:

[[14  7 -1]
 [ 2 -4  1]
 [11  2  0]]



In [64]:
def cofactor(A):
    C = np.zeros_like(A)
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            m = minor(A,i+1,j+1)
            cofactor = ((-1)**(i+j)) * LUdet(m)
            C[i,j] = cofactor 
    return C 

def detInv(A):
    C = cofactor(A)
    return C.T/LUdet(A)
    


# Question 2 
print("Determinant of A is: ", end="\n\n")
print(LUdet(A),end="\n\n")
print("The property holds because both matrices are equal: ",end="\n\n")
print(np.isclose(B@cofactor(B).T,LUdet(B)*np.identity(4)), end="\n\n")

# Question 3: 
print("The inverse of A calculated via detInv() is: ", end="\n\n")
print(detInv(A),end="\n\n")

# Question 4
print("The inverse of A calculated via detInv(A) equal to np.linalg.inv(A): ", end="\n\n")
print(np.isclose(np.linalg.inv(A),detInv(A)),end="\n\n")

#Question 5
print("Time to compute the inverse of A via detInv():",end="\n\n")
%timeit detInv(B)
print("\n")
print("Time to compute the inverse of A via rowredinv():",end="\n\n")
%timeit rowredinv(B)

Determinant of A is: 

1.0000000000000018

The property holds because both matrices are equal: 

[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]

The inverse of A calculated via detInv() is: 

[[14.  2. 11.]
 [ 7. -4.  2.]
 [-1.  1.  0.]]

The inverse of A calculated via detInv(A) equal to np.linalg.inv(A): 

[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]

Time to compute the inverse of A via detInv():

1.56 ms ± 20.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



Time to compute the inverse of A via rowredinv():

107 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Homework Question 3

It is often said that determinants are one of the least useful ways to do numerical computation with matrices. Do some research online and read a paper or two, then write a paragraph summarizing why this is the case. Include references.

Based on our research, determinants were extremely important in the early days of mathematics because they were used to explain mathematical theory and gain a greater understanding of linear algebra as a whole. However, it was soon realized that calculating determinants is extremely computation heavy. As most computer science students are aware, the speed of an algorithm is measured by finding how the time taken by the computer grows as the size of its input data set grows. In other words, the speed of an algorithm is measured by it’s asymptotic growth with respect to the size of its input. Computing determinants has a runtime of O(n!), which is one of the slowest algorithms possible. For example, if one wanted to calculate the determinant of a 10 x 10 matrix, it would have over 10! (3,628,800) operations to compute, making it extremely slow and inefficient. There has been some research in trying to make calculating determinants faster, such as via “dynamic determinant computation” which can do some determinant calculations in cubic runtime, but this is a relatively new breakthrough. Likewise, a cubic runtime is also extremely slow and there are numerous other methods that could be used without taking determinants that are much faster. 

References: 
1) https://en.wikibooks.org/wiki/Linear_Algebra/Topic:_Speed_of_Calculating_Determinants
2) https://arxiv.org/pdf/1206.7067.pdf 